# Logistic Regression
---
                                                            written by Yang, Soyoung
                                                                     2017.07.03 v1.0
                                                                     2017.07.28 v1.1
                                                                     2017.07.29 v1.2
                                                                     2017.08.09 v1.3

__위젯(widget)__으로 인풋 샘플들과 매개 변수(hyperparameter, ex. 학습률, epoch의 횟수)를 조정하고

__텐서플로우(tensorflow)__를 사용해 로지스틱회기(logistic regression)의 방법과 과정을 살펴본다.


#### steps
1. importing
2. Data generation
3. Model parameters
4. Tensorflow : Graphing
5. Tensorflow : Session
6. Plotting



---
## 로지스틱 회기 Logistic regression

- 로지스틱 회기는 집단을 구분하는 분류기(classifier)이다. 
- 예를 들어 0과 1인 클래스를 구분할 때, 샘플 x의 예측 값이 0.5 이상이라면 1로, 0.5 미만이라면 0으로 구분한다. 한 종양이 악성인지 양성인지를 판단 할 때 나이나 성별, 종양의 크기를 고려하여 예측 값이 악성(1)인지 양성(0)인지를 판단하는 것과 같은 원리이다.

<img src="imgs/lda_binary.png" style="width: 500px;">


### 1) cost function

- 선형회기에서는 전역 최소점(global minima)만 존재하기 때문에 [SSE](https://en.wikipedia.org/wiki/Residual_sum_of_squares)로 기울기 하강법(gradient descent)을 이용해도 괜찮았다. 하지만 로지스틱 회기의 경우 여러 지역 최소점들(local minima)이 존재하므로, SSE대신 __log__ 를 cost function으로 사용한다. 

- 목표값 y가 1일때 예측값 h(x)이 0에 가까울 수록 에러는 매우 커지고 h(x)가 1에 가까워질 수록 에러는 작아진다. 목표값 y가 0일때는 그 반대가 된다. 이 cost function을 한줄로 쓰면 J(theta)와 같다. 

<img src="imgs/logisticReg_cost.png" style="width: 500px;">



### 2) regularization

<img src="imgs/logisticReg_overfit.png" style="width: 600px;">


- 하지만 분류기가 학습데이터에만 과하게 트레이닝 될 경우 세번째 그림과 같은 과적합(over fitting)이 발생할 수 있다. 이때 분류기는 학습 데이터에서만 정확도가 높고, 일반성(generality)이 매우 떨어지기 때문에 새로 들어오는 데이터를 적절히 처리하기 어렵다. 마치 자잘한 일에 신경쓰느라 큰 일(빅피쳐)을 놓치는 것과 같다. 
- 과적합을 줄이는 방법은 학습 데이터를 크게 하거나 특징(feature)을 줄이는 방법이 있다. 일반적으로는 cost function에 정규화식(regularization)을 넣는다. 


<img src="imgs/regularization.png" style="width: 600px;">


- 기존 cost 뒤에 정규화식을 더해줘서 다차항의 구부러진 모델을 쭉 펴준다. 즉 모델을 더 간단하고 __일반적__으로(핑크색) 만든다. 람다는 정규화식 파라미터(regularization parameter/strength)로, 람다가 너무 클 경우 모든 feature가 0으로 가서 평평해지고, 람다가 너무 작을 경우 정규화되지 않는다. 따라서 적절한 람다를 설정하는 것이 중요하다.

- tensorflow 코드로는 간단하게 구현할 수 있다. :  ```l2reg = 0.001 * tf.reduce_sum(tf.square(W))``` 이 식을 cost에 더한 다음에 cost를 최소화 시킨다.

- 로지스틱 회기의 경우 정규화된 cost는 아래와 같다.


<img src="imgs/logisticReg_regcost.png" style="width: 700px;">

- 여기에서 alpha는 학습률, learning rate라고 하는데 경사하강법(gradient descent)을 할 때 한 번 내려가는 걸음의 크기를 조정한다. 보폭이 너무 크면 최적점에 도달하지 못하고 오히려 바깥으로 벗어나는 overshooting이 일어나고, 보폭이 너무 작으면 학습 시간이 오래걸린다. 보통 0.001을 사용한다.


---

## 1. importing

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import *

## 2. Data generation
specify
- sample size per class -> will be doubled
- interval for feature 1
- interval for feature 2

In [2]:
## 2. data generate

widget_size_per_class = widgets.IntSlider(value=10, min=10, max=100, step=10,
                                          description='sample size per class :', readout=True)
widget_interval_feature_1 = widgets.IntRangeSlider(value=[10,15], min=0, max=20,step=1,
                                                   description='Interval of feature1 (b):', readout=True)
widget_interval_feature_2 = widgets.IntRangeSlider(value=[5,10], min=0, max=20, step=1,
                                                   description='Interval of feature2 (r): ', readout=True)


# callback for widgets
def merged_widget_make_data(sample_size_per_class, interval_feature_1, interval_feature_2):
    total_sample_size = 2 * sample_size_per_class
    
    # make data with random integer
    class0_x1 = np.random.randint(interval_feature_1[0], interval_feature_1[1], sample_size_per_class)
    class0_x2 = np.random.randint(interval_feature_2[0], interval_feature_2[1], sample_size_per_class)
    class1_x1 = np.random.randint(interval_feature_2[0], interval_feature_2[1], sample_size_per_class)
    class1_x2 = np.random.randint(interval_feature_1[0], interval_feature_1[1], sample_size_per_class)
    
    # plot
    plt.plot(class0_x1, class0_x2, 'bx', class1_x1, class1_x2, 'rx')
    plt.show()
    
    # make samples with label[x1, x2, label]
    # combine them as data, and shuffle data as randomization
    class0_samples_with_label = np.column_stack([class0_x1, class0_x2, np.zeros(sample_size_per_class)])
    class1_samples_with_label = np.column_stack([class1_x1, class1_x2, np.ones(sample_size_per_class)])
    data = np.concatenate([class0_samples_with_label, class1_samples_with_label], axis=0)
    np.random.shuffle(data)
    
    ## split data into train, test set
    # 70% is train data, and last 30% is test data
    criteria = int(round(total_sample_size*0.7))
    
    train_data = data[:criteria,:]
    train_steps = criteria
    
    test_data = data[criteria:,:]
    test_steps = total_sample_size - criteria
    
    return(train_data, test_data, int(train_steps), int(test_steps), int(total_sample_size))



merged_widget_data = interactive(merged_widget_make_data, sample_size_per_class = widget_size_per_class,
                                 interval_feature_1=widget_interval_feature_1, interval_feature_2=widget_interval_feature_2)

display(merged_widget_data)

## 3. Model parameters
- learning rate
- epoch size

In [3]:
widget_lr = widgets.FloatSlider(value=0.001, min=0.001, max=1.000, step=0.001,\
                                description='learning rate : ', readout=True, readout_format='0.3f')
widget_epoch_size = widgets.IntSlider(value=200, min=100, max=1000, step=100,\
                                      description='epoch size : ', readout=True)

widget_init = widgets.ToggleButton(value=False, description='Initialize', disabled=False,
                                   button_style='info',icon='check')

# callback for lr, epoch
def merged_widget_model_param(lr, epoch):
    return(lr, epoch)

# callback for initialize button
def initialize(value):
    if value: # run when value==true
        widget_lr.value=0.001
        widget_epoch_size.value=200

merged_widget_for_model = interactive(merged_widget_model_param, lr = widget_lr, epoch = widget_epoch_size)
initial_lr_epoch = interactive(initialize, value=widget_init)

display(merged_widget_for_model, initial_lr_epoch)

## 4. tensorflow : Graphing

In [4]:
## gather data and parameters from widget
train_data, test_data, train_steps, test_steps, total_size = merged_widget_data.result
lr, epoch_size = merged_widget_for_model.result

In [5]:
## Graphing

# placeholder <- where input will take place
X = tf.placeholder(name='input_data', shape=[1,2], dtype=tf.float32)
Y = tf.placeholder(name='target_data', shape=[1,1], dtype=tf.float32)

# Variable
w = tf.Variable(initial_value=tf.random_uniform([2,1], minval=0, maxval=5, dtype=tf.float32),\
                name='weight', dtype=tf.float32)
b = tf.Variable(initial_value=tf.random_uniform([1,1], minval=0, maxval=5, dtype=tf.float32),\
                name='bias', dtype=tf.float32)

# decision_boundary = X*w (1,2 * 2,1) + b
decision_boundary = tf.add(tf.matmul(X,w),b)
# logistic regression take output using sigmoid
Y_predict = tf.sigmoid(decision_boundary)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = decision_boundary, labels = Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr).minimize(loss)

# accuracy checking
predicted_label = tf.cast(Y_predict >= 0.5, tf.float32)
correct = tf.cast(tf.equal(predicted_label, Y),tf.float32)

# initializer
init = tf.global_variables_initializer()

## 5. Session.run()

In [6]:
w_stack = np.empty([epoch_size,2])
b_stack = np.empty([epoch_size,1])

with tf.Session() as sess:
    sess.run(init)
    
    ## training
    for epoch in range(epoch_size):
        total_loss=0
        for step in range(train_steps):
            x = train_data[step,0:2].reshape(1,2) # sample
            y = train_data[step,2].reshape(1,1)   # target data
            
            _, local_loss = sess.run([optimizer, loss], feed_dict={X:x, Y:y}) 
            total_loss += local_loss              # optimizer is operator, so doesnt return any value
            
        total_loss = total_loss/train_steps
        if epoch%50==0:
            print("epoch {}, loss : {:0.4f}".format(epoch, total_loss))
            
        current_w, current_b = sess.run([w,b])    # w, b are tensor variable, should be run to get their value
        w_stack[epoch] = current_w.reshape(1,2)   # w.shape=[2,1] so reshape w to [1, 2] stack in list
        b_stack[epoch] = current_b.reshape(1,1)
        
    print('Training done. \n')
    
    ## check accuracy with test set
    print("Target_label | predicted_label | correct?")
    for step in range(test_steps):
        x = test_data[step,0:2].reshape(1,2)
        y = test_data[step,2].reshape(1,1)
        step_predicted_label, step_correct = sess.run([predicted_label, correct], feed_dict={X:x,Y:y})
        print("{} | {} | {}".format(y, step_predicted_label, step_correct))

epoch 0, loss : 14.6675
epoch 50, loss : 0.2266
epoch 100, loss : 0.2220
epoch 150, loss : 0.2204
Training done. 

Target_label | predicted_label | correct?
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 1.]] | [[ 0.]] | [[ 0.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 1.]] | [[ 0.]] | [[ 0.]]
[[ 1.]] | [[ 1.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]
[[ 0.]] | [[ 0.]] | [[ 1.]]


## 6. Plotting

- [delete rows containing non-numberic](https://stackoverflow.com/questions/11453141/how-to-remove-all-rows-in-a-numpy-ndarray-that-contain-non-numeric-values)
- [decision boundary, regression](https://stackoverflow.com/questions/20045994/how-do-i-plot-the-decision-boundary-of-a-regression-using-matplotlib)
- [plot decision boundary](https://stackoverflow.com/questions/42704698/logistic-regression-plotting-decision-boundary-from-theta)



In [7]:
# pre-processing
wb_stack = np.column_stack((w_stack, b_stack))

wb_stack = wb_stack[~np.isnan(wb_stack).any(axis=1)]
    # remove all rows that containing non-numberic values
    # ```.any(axis=1)``` reduces an m*n array to n with an logical or operation on the whole rows
w_stack = wb_stack[:,0:2]
b_stack = wb_stack[:,2]

origin_data = np.concatenate([train_data, test_data])

In [8]:
# widgets
widget_weight_1 = widgets.SelectionSlider(options=w_stack[:,0].tolist(), value=w_stack[0,0], \
                                          description='weight1 :', readout=True, readout_format='0.3f')
widget_weight_2 = widgets.SelectionSlider(options=w_stack[:,1].tolist(), value=w_stack[0,1], \
                                          description='weight2 :', readout=True, readout_format='0.3f')
widget_bias = widgets.SelectionSlider(options=b_stack.tolist(), value=b_stack[0],\
                                      description='bias : ', readout=True, readout_format='0.3f')

widget_epoch_step = widgets.IntSlider(min=0, max=epoch_size-1, value=0, description="epoch step :", readout=True)

# callback
def merged_widget_plot(w1, w2, b):
    print('\tuntrained ---------> trained')
    x1, x2, y = origin_data[:,0], origin_data[:,1], origin_data[:,2]
    class0 = np.where(y==0)
    class1 = np.where(y==1)
    plt.plot(x1[class0],x2[class0], 'bo', x1[class1], x2[class1], 'ro')
    
    # w1*x1 + w2*x2 + b = 0
    # x2 = -(b + w1*x1)/w2
    X = np.linspace(0,20, 50)
    Y = -(b + w1*X)/w2
    plt.plot(X,Y)
    plt.xlabel("feature1 (b)")
    plt.ylabel("feature2 (r)")
    plt.axis([-1, 20, -1, 20])
    plt.show()
    
def merged_widget_process(epoch):
    x1, x2, y = origin_data[:,0], origin_data[:,1], origin_data[:,2]
    class0 = np.where(y==0)
    class1 = np.where(y==1)
    plt.plot(x1[class0],x2[class0], 'bo', x1[class1], x2[class1], 'ro')
    
    w1, w2, b = w_stack[epoch,0], w_stack[epoch,1], b_stack[epoch]
    print("\tw1 : {:0.4f}, w2 : {:0.4f}, b : {:0.4f}".format(w1,w2,b))
    
    X = np.linspace(0,20, 50)
    Y = -(b + w1*X)/w2
    plt.plot(X,Y)
    plt.xlabel("feature1 (b)")
    plt.ylabel("feature2 (r)")
    plt.axis([-1, 20, -1, 20])
    plt.show()

# interactive
merged_wb_result_plot = interactive(merged_widget_plot, w1=widget_weight_1,w2=widget_weight_2, b=widget_bias)
show_running_process = interactive(merged_widget_process, epoch=widget_epoch_step)


display(merged_wb_result_plot, show_running_process)